In [1]:
from itertools import chain
import pandas as pd
import time
import matplotlib.pyplot as plt
max_it = 10
from misc import vectorSchRepresentation, feasible_pairs, feasible_recharge, visualizeResult, visualizeSolution, drawBusTrips
from chs import constructiveScheduler
from sa import annealing, get_total_gap

In [2]:
filename = f"./bus_555_0_600_2024-7-26.xlsx"
NTrips = 499 #trips
NTerms = 2 #terminals
NCS = 2 #recharging station
CHARGING_TIME = 100 #minutes
NRechargeCycle = 3 #cycle
D_MAX = 150 #minutes of operation
DEPOT = 1 #depot
df = pd.read_excel(filename, sheet_name="main", usecols="B:J", nrows=NTrips+NCS+DEPOT)
term = pd.read_excel(filename, sheet_name="main", usecols="L:M", skiprows=9, nrows=NTerms)
gamma = pd.read_excel(filename, sheet_name="gamma", usecols="B:SE", skiprows=0, nrows=NTrips+DEPOT)
delta = pd.read_excel(filename, sheet_name="delta", usecols="B:SG", skiprows=0, nrows=NTrips+DEPOT+NCS)
phi = pd.read_excel(filename, sheet_name="phi", usecols="C:SF", nrows=(NTrips+DEPOT)*NCS)
df2 = df.set_index('trip_id')
print(df2.tail())
gamma = gamma.to_numpy()
print(gamma.shape)
delta = delta.to_numpy()
print(delta.shape)
phi = phi.to_numpy()
print(phi.shape)

                           type shape_id  direction_id  duration  dep_time  \
trip_id                                                                      
29462396-LCBS 24_25-37102  trip  5550277           0.0        38       570   
29462655-LCBS 24_25-37102  trip  5550277           0.0        54       465   
29462656-LCBS 24_25-37102  trip  5550340           1.0        42       520   
CS1                          cs        -           NaN        15         0   
CS2                          cs        -           NaN        20         0   

                           arr_time     dep_term     arr_term  
trip_id                                                        
29462396-LCBS 24_25-37102       608   Terminal_1  Terminal_13  
29462655-LCBS 24_25-37102       519   Terminal_1  Terminal_13  
29462656-LCBS 24_25-37102       562  Terminal_13   Terminal_1  
CS1                            1440            -            -  
CS2                            1440            -            -  
(498,

In [3]:
df2['ID'] = [i for i in range(len(df2))]
df2

,type,shape_id,direction_id,duration,dep_time,arr_time,dep_term,arr_term,ID
trip_id,,,,,,,,,
DEPOT,depot,-,NaN,0,0,1440,-,-,0
29081731-LCBS 24_25-36695,trip,5550277,0.0,35,360,395,Terminal_1,Terminal_13,1
29081732-LCBS 24_25-36695,trip,5550340,1.0,41,400,441,Terminal_13,Terminal_1,2
29081733-LCBS 24_25-36695,trip,5550277,0.0,35,450,485,Terminal_1,Terminal_13,3
29081734-LCBS 24_25-36695,trip,5550340,1.0,41,490,531,Terminal_13,Terminal_1,4
...,...,...,...,...,...,...,...,...,...
29462396-LCBS 24_25-37102,trip,5550277,0.0,38,570,608,Terminal_1,Terminal_13,495
29462655-LCBS 24_25-37102,trip,5550277,0.0,54,465,519,Terminal_1,Terminal_13,496
29462656-LCBS 24_25-37102,trip,5550340,1.0,42,520,562,Terminal_13,Terminal_1,497


In [4]:
lendf = len(df2)-2
### Add extra parameter that allows e-Bus to complete schedule that is not continuous (aka the start node does not have to be the end node of the previous)
## Defining the deadhead parameter from Terminals
deadheads = {
    'Terminal A': {'Terminal B': 25, 'Terminal C': 30, 'Terminal D': 35},
    'Terminal B': {'Terminal A': 25, 'Terminal C': 28, 'Terminal D': 40},
    'Terminal C': {'Terminal A': 30, 'Terminal B': 28, 'Terminal D': 33},
    'Terminal D': {'Terminal A': 35, 'Terminal B': 40, 'Terminal C': 33},
}
D_MAX = 150
CHARGING_TIME = 100

charging_stations = {
    (lendf)+1: {'trip_id': 'CS1', 'type': 'cs', 'duration': 15,  'dep_time': 0, 'arr_time': 1440, 'dep_term': '-', 'arr_term': '-'},
    (lendf)+2: {'trip_id': 'CS2', 'type': 'cs', 'duration': 20,  'dep_time': 0, 'arr_time': 1440, 'dep_term': '-', 'arr_term': '-'},
}

# TERMINALS AND MINIMUM INTERVALS BETWEEN TRIPS
terminals = {
    'Terminal_1': {'max_interval': 15},
    'Terminal_13': {'max_interval': 15},
#     'Terminal C': {'max_interval': 15},
#     'Terminal D': {'max_interval': 15}
}

### Add extra parameter that allows e-Bus to complete schedule that is not continuous (aka the start node does not have to be the end node of the previous)
## Defining the deadhead parameter from Terminals
deadheads = {
    'Terminal_1': {'Terminal_13': 25},
    'Terminal_13': {'Terminal_1': 25},
#     'Terminal C': {'deadhead_time': 15},
#     'Terminal D': {'deadhead_time': 10},
}

In [5]:
df = df.head(lendf)
print(df.tail())
print(feasible_pairs(df, terminals=terminals))
arcs = feasible_pairs(df, terminals=terminals)
print(charging_stations)
## Creating Phi and Delta
print(feasible_recharge(df, arcs, recharge=charging_stations))
recharge_arcs = feasible_recharge(df, arcs, recharge=charging_stations)
cs = list(charging_stations.keys())

cs_df = pd.DataFrame.from_dict(charging_stations, orient='index')
all_schs = pd.concat([df, cs_df], ignore_index=True)
print(all_schs)
durations = {idx: {'duration': all_schs.loc[idx, 'duration']} for idx in all_schs.index if idx != 0}

                       trip_id  type shape_id  direction_id  duration  \
493  29462325-LCBS 24_25-37102  trip  5550277           0.0        50   
494  29462326-LCBS 24_25-37102  trip  5550340           1.0        42   
495  29462396-LCBS 24_25-37102  trip  5550277           0.0        38   
496  29462655-LCBS 24_25-37102  trip  5550277           0.0        54   
497  29462656-LCBS 24_25-37102  trip  5550340           1.0        42   

     dep_time  arr_time     dep_term     arr_term  
493       495       545   Terminal_1  Terminal_13  
494       548       590  Terminal_13   Terminal_1  
495       570       608   Terminal_1  Terminal_13  
496       465       519   Terminal_1  Terminal_13  
497       520       562  Terminal_13   Terminal_1  
    trip_id                    trip_id   type shape_id  direction_id  \
0         0                      DEPOT  depot        -           NaN   
1         1  29081731-LCBS 24_25-36695   trip  5550277           0.0   
2         2  29081732-LCBS 24_25-

{499: {'trip_id': 'CS1', 'type': 'cs', 'duration': 15, 'dep_time': 0, 'arr_time': 1440, 'dep_term': '-', 'arr_term': '-'}, 500: {'trip_id': 'CS2', 'type': 'cs', 'duration': 20, 'dep_time': 0, 'arr_time': 1440, 'dep_term': '-', 'arr_term': '-'}}


TypeError: feasible_recharge() missing 1 required positional argument: 'terminals'

In [ ]:
# ######################################################## 1CS #####################################################################
# recharge_1cs_arcs = feasible_recharge(trips_df, cs_deadheads, recharge=charging_station, terminals=terminals)
# cs_ids = set(list(charging_station.keys()))
# all_schedules_1cs = {**trips, **charging_station}
# all_schs_1cs = pd.DataFrame.from_dict(all_schedules_1cs, orient='index')
# all_schs_1cs['ID'] = range(len(all_schs_1cs))
# durations_1cs = {idx: {'duration': all_schs_1cs.loc[idx, 'duration']} for idx in all_schs_1cs.index if idx != 0}

# ################################################################### CHS-SA ###########################################################################################
# ####### CHS
# import time
# start_time = time.time()
# print(f"{time.ctime()}")
# schedules_tab, schedules_10_1 = constructiveScheduler(all_schs_1cs, arcs, recharge_1cs_arcs, cs_ids)
# solution = vectorSchRepresentation(schedules_10_1)
# end_time = time.time()
# print(f"{time.ctime()}\nTime elapse to compute the solution = {(end_time - start_time)} seconds")
# print(f"number of buses = {len(schedules_10_1)}")
# print(schedules_10_1)
# g_T10CS1_time = end_time-start_time

# ####### SA
# print('-'*100)
# print("starting simulated annealing....")
# start_time = time.time()
# # test_new_schedule, test_cost, test_cost_diffs, test_temp, test_it, test_costs, test_solutionspaces, test_best
# new_schedule_10_1, cost_10_1, cost_diffs_10_1, temp_10_1, it_10_1, costs_10_1, solution_spaces_10_1, best_costs_10_1 = annealing(solution, all_schs_1cs, arcs, recharge_1cs_arcs)
# end_time = time.time()
# T10CS1_time = (end_time-start_time) + g_T10CS1_time
# print(f"{time.ctime()}\nTime elapse to compute the solution = {(end_time - start_time)} seconds")
# print(f"prev_schedule = {solution} with number of buses = {len(solution)}... \nnext_schedule = {new_schedule_10_1} with number of buses = {len(new_schedule_10_1)}")
# print(f"Number of buses previous = {len(schedules_10_1)}... new = {len(new_schedule_10_1)} ")
# print(f"Prev: {vectorSchRepresentation(schedules_10_1)}\nNew: {new_schedule_10_1}")
# print(f"Total Gap among buses=> prev = {get_total_gap(solution, all_schs_1cs, recharge_1cs_arcs)}... new = {get_total_gap(new_schedule_10_1, all_schs_1cs, recharge_1cs_arcs)}")


import time
start_time = time.time()
print(f"{time.ctime()}")
schedules_tab, schedules = constructiveScheduler(all_schs, arcs, recharge_arcs, set(cs))
solution = vectorSchRepresentation(schedules)
end_time = time.time()
print(f"{time.ctime()}\nTime elapse to compute the solution = {(end_time - start_time)} seconds")
print(f"number of buses = {len(schedules)}")
print(schedules)
print('-'*100)